In [1]:
import os
import pandas as pd
import numpy as np
import operator
import datetime as dt

In [2]:
h1file = os.path.join('..','rawData',"H1N1-Data.csv")
rawh1 = pd.read_csv(h1file,header= 0,encoding= 'ISO-8859-1')

cases_df = pd.DataFrame({'Virus': [], 'Country': [], 'Latitude': [], 'Longitude': []})
deaths_df = pd.DataFrame({'Virus': [], 'Country': [], 'Latitude': [], 'Longitude': []})
base_date = dt.date(2009, 5, 23)
last_day = -1

In [3]:
rawh1case = rawh1
rawh1case = rawh1case.reindex(index = rawh1case.index[::-1])
rawh1case = rawh1case.reset_index(drop = True)
rawh1case

,Country,Cases,Deaths,Update Time
0,Grand Total,12022,86,5/23/2009 8:00
1,United States of America,6552,9,5/23/2009 8:00
2,United Kingdom,117,0,5/23/2009 8:00
3,Turkey,2,0,5/23/2009 8:00
4,Thailand,2,0,5/23/2009 8:00
...,...,...,...,...
1817,Austria,19,0,7/6/2009 9:00
1818,Australia,5298,10,7/6/2009 9:00
1819,Argentina,2485,60,7/6/2009 9:00
1820,Antigua and Barbuda,2,0,7/6/2009 9:00


In [4]:
for index, row in rawh1case.iterrows():
    country = row['Country']
    cases = row['Cases']
    deaths = row['Deaths']
    update_time = row['Update Time']
    
    timedelta = (dt.datetime.strptime(update_time, '%m/%d/%Y %H:%M').date()-base_date).days
    column = 'Day ' + str(timedelta)
    days_passed = timedelta - last_day
    if days_passed > 0:
        while days_passed > 1:
            cases_df['Day ' + str(timedelta - days_passed + 1)] = cases_df['Day ' + str(last_day)]
            deaths_df['Day ' + str(timedelta - days_passed + 1)] = deaths_df['Day ' + str(last_day)]
            days_passed = days_passed - 1
        cases_df[column] = ''
        deaths_df[column] = ''
    #we must do a large amount of renaming in order to match the rest of our data
    if '    ' in country:
        country = ''
    if 'Grand Total' in country:
        country = ''
    if 'United States' in country:
        country = 'United States'
    if 'Korea' in country:
        country = 'South Korea'
    if 'Iran' in country:
        country = 'Iran'
    if 'Viet Nam' in country:
        country = 'Vietnam'
    if 'West Bank' in country:
        country = 'West Bank'
    if 'Jersey' in country:
        country = 'Jersey'
    if 'Guadaloupe' in country:
        country = 'Guadeloupe'
    if 'Isle of Man' in country:
        country = 'Isle of Man'
    if 'Netherlands Antilles' in country:
        country = 'Netherlands Antilles'
    if 'Saint Martin' in country:
        country = 'Netherlands Antilles'
    if 'Brunei' in country:
        country = 'Brunei'
    if 'Bosnia' in country:
        country = 'Bosnia and Herzegovina'
    if "Cote d'Ivoire" in country:
        country = "Côte d'Ivoire"
    if 'Cap Verde' in country:
        country = 'Cape Verde'
    if 'Guernsey' in country:
        country = 'Guernsey'
    if 'Aruba' in country:
        country = 'Aruba'
    if 'Myanmar' in country:
        country = 'Myanmar [Burma]'
    if 'Macedonia' in country:
        country = 'Macedonia [FYROM]'
    if 'Virgin' in country:
        #we use the U.S. Virgin Islands as geographically they are in the center
        country = 'U.S. Virgin Islands'
    if 'Cook' in country:
        country = 'Cook Islands'
    if 'FOC' in country:
        country = country[:-5]
    if 'UKOT' in country:
        country = country[:-6]
    country = ''.join(i for i in country if i!='*')
    country = country.strip()
    
    
    last_day = timedelta
    if (country!='') and (cases!='') and (deaths!=''):
        #add data to table, adding rows if necessary
            if country not in cases_df['Country'].tolist():
                new_country_cases = ['H1N1', country,0,0]
                new_country_deaths = ['H1N1', country,0,0]
                count = 0
                while count < timedelta:
                    new_country_cases.append(int(0))
                    new_country_deaths.append(int(0))
                    count += 1
                new_country_cases.append(int(cases))
                new_country_deaths.append(int(deaths))
                cases_df.loc[len(cases_df)] = new_country_cases
                deaths_df.loc[len(deaths_df)] = new_country_deaths
            else:
                cases_df.loc[cases_df['Country'] == country, [column]] = int(cases)
                deaths_df.loc[deaths_df['Country'] == country, [column]] = int(deaths)

In [5]:
#some countries have recorded cases, but those cases are found to not be H1N1. In this case,
#these countries won't come up again in the cumulative data, leaving empty cells. We are not
#interested in these for our analysis, so we drop these (only the ones that never come back)
cases_df = cases_df[cases_df['Day 44'] != '']

In [6]:
#fortunately, deaths are never mistaken to be caused by something other than H1N1, so we
#don't have to worry about that here. We instead have the issue where 0 deaths will be
#recorded even if there are cases, so we must remove any row that finishes with 0 deaths
deaths_df = deaths_df[deaths_df['Day 44'] != '']
deaths_df = deaths_df[deaths_df['Day 44'] != 0]

In [7]:
#we read in our coords csv to add our lat and long coordinates
path = os.path.join('..','rawdata','coords.csv')
coords_df = pd.read_csv(path)

In [8]:
def get_coords(row):
    country = row[1]
    if country in coords_df['Country'].tolist():
        lat = float(coords_df[coords_df['Country'] == country]['Latitude'])
        long = float(coords_df[coords_df['Country'] == country]['Longitude'])
        return (lat,long)
    else:
        print(country)
        return (0,0)

In [9]:
cases_df['Latitude'] = cases_df.apply(lambda row: get_coords(row)[0], axis=1)
cases_df['Longitude'] = cases_df.apply(lambda row: get_coords(row)[1], axis=1)
deaths_df['Latitude'] = deaths_df.apply(lambda row: get_coords(row)[0], axis=1)
deaths_df['Longitude'] = deaths_df.apply(lambda row: get_coords(row)[1], axis=1)

West Bank
West Bank


In [10]:
path = os.path.join('..','cleanData','H1N1_cases.csv')
cases_df.to_csv(path)
path = os.path.join('..','cleanData','H1N1_deaths.csv')
deaths_df.to_csv(path)